In [1]:
import os

os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasetsforecast.m3 import M3

from models.auto.AutoTimeMoe import AutoTimeMoe
from neuralforecast.tsdataset import TimeSeriesDataset

c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-05 22:19:32,894	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-05 22:19:33,121	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
# load m3
df = M3.load(
            directory="C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m3",
            group="Monthly")[0]

def train_test_split(df: pd.DataFrame, horizon: int):
    """Split the dataframe into training and test sets by horizon."""
    groups = df.groupby('unique_id')
    train_list, test_list = [], []
    for _, group_df in groups:
        group_df = group_df.sort_values('ds')
        train_list.append(group_df.head(-horizon))
        test_list.append(group_df.tail(horizon))
    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)
    return train_df, test_df

# df['ds'] = pd.to_datetime(df['ds']).astype(int)

y_train_df, y_test_df = train_test_split(df, horizon=18)

dataset, *_ = TimeSeriesDataset.from_df(y_train_df)

c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\datasetsforecast\m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


In [4]:
model = AutoTimeMoe(h=18, num_samples=20, backend="optuna")

model.fit(dataset)

[I 2025-03-05 22:19:43,803] A new study created in memory with name: no-name-9bf44ea0-8c7b-4333-bcae-1f1f761f5535
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\neuralforecast\common\_base_auto.py:291: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  v = trial.suggest_loguniform(k, v.lower, v.upper)
INFO:lightning_fabric.utilities.seed:Seed set to 11
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type                  | Params | Mode 
---------------------------------

Epoch 83:  33%|███▎      | 2/6 [00:00<00:01,  2.66it/s, v_num=157, train_loss_step=2.45e+3, train_loss_epoch=2.91e+3, valid_loss=6.94e+4]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=500` reached.


Epoch 83:  33%|███▎      | 2/6 [00:00<00:01,  2.64it/s, v_num=157, train_loss_step=2.45e+3, train_loss_epoch=2.91e+3, valid_loss=6.94e+4]


[I 2025-03-05 22:20:09,572] Trial 0 finished with value: 69037.4921875 and parameters: {'hidden_size': 64, 'n_head': 2, 'learning_rate': 0.001776203019728038, 'scaler_type': 'robust', 'max_steps': 500, 'batch_size': 256, 'windows_batch_size': 128, 'intermediate_size': 20000, 'random_seed': 11, 'num_experts_per_tok': 1, 'num_experts': 4, 'input_size': 72, 'step_size': 1}. Best is trial 0 with value: 69037.4921875.
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\neuralforecast\common\_base_auto.py:291: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  v = trial.suggest_loguniform(k, v.lower, v.upper)
INFO:lightning_fabric.utilities.seed:Seed set to 10
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU core

...

[I 2025-03-05 22:57:30,028] Trial 19 finished with value: 67981.5546875 and parameters: {'hidden_size': 128, 'n_head': 8, 'learning_rate': 0.0032655891308713755, 'scaler_type': 'standard', 'max_steps': 1000, 'batch_size': 32, 'windows_batch_size': 128, 'intermediate_size': 10000, 'random_seed': 5, 'num_experts_per_tok': 4, 'num_experts': 8, 'input_size': 18, 'step_size': 1}. Best is trial 14 with value: 55420.65625.
INFO:lightning_fabric.utilities.seed:Seed set to 9
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type                  | Params | Mode 
---------------------------------------------------------------
0 | loss         | MAE         

Epoch 22:  22%|██▏       | 10/45 [00:01<00:03,  9.91it/s, v_num=177, train_loss_step=1.89e+5, train_loss_epoch=2.1e+5] 

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 22:  22%|██▏       | 10/45 [00:01<00:03,  9.88it/s, v_num=177, train_loss_step=1.89e+5, train_loss_epoch=2.1e+5]


AutoTimeMoe

In [ ]:
# get best model config
test_dataset, *_ = TimeSeriesDataset.from_df(y_test_df)

y_hat = model.predict(dataset=test_dataset)

y_hat